In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
from column_map import column_map

In [7]:
column_map('Ambassador_CW').values()

dict_values(['Encounter_Timestamp', 'Encounter_DateTime', 'Property', 'Location_AddDesc', 'CityAgency', 'FirstName_1', 'LastName_1', 'FirstName_2', 'LastName_2', 'FirstName_3', 'LastName_3', 'Encounter_Type', 'SD_PatronsComplied', 'SD_PatronsNoComply', 'SD_Amenity', 'SD_PDContact', 'SD_Comments', 'Closed_Amenity', 'Closed_PatronCount', 'Closed_Approach', 'Closed_Outcome', 'Closed_PDContact', 'Closed_Comments', 'Borough'])

In [ ]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)